In [102]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.svm import SVR
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from random import randint

In [50]:
#colunas que escolhi para dados escolares
col_escol = ['CODMEC','ANO','TIPOESC','DEP','Qtde_Docentes','NUM_FUNC','MERENDA','CLFUND',
            'ALFUND','TOTALEJA','TOT_EE','TOTAL_ALUNOS','CL_CRECHE','AL_CRECHE','CL_PRE','AL_PRE','CLEM','ALEM']
col_rendi = ['CODMEC','EFABAND9ANOS8S','EFABAND9ANOS9S','EFABAND7S','EFABAND8S','TOTALEF_ABAND','TOTALEF_APROV',
             'TOTALEF_REPROV','TOTAL_EM_ABAND','TOTAL_EM_REPROV','CONCLUINTES_EM','EPABAND','EPREPROV']
dados_escolares = pd.read_csv('DadosEscolares1996-2015.csv', encoding = 'latin1', usecols = col_escol)
rendimento = pd.read_csv('RendimentoEscolar2000-2015.csv', encoding = 'latin1', usecols = col_rendi)
esc_rmsp = pd.read_csv('ESC2013_RMSP_CEM.csv', encoding = 'latin1')
nota = pd.read_csv('ENEM2015.csv')
#como eu abri todas as colunas do esc_rmsp, agora vou retirar as que não desejo usar
col_rmsp = ['ID', 'LONGITUDE', 'LATITUDE', 'CODESCTX','NOMEESC', 'NOMEMUN','CODMUN', 'NOMDIST', 'CODDIST', 'COD_SC',
           'BAIRRO','CEP','END_ESC','NUM_ESC','DDD','TELEFONE','ENS_REG','REG_IN_CR','REG_IN_PE','REG_FU_8A',
           'REG_FU_9A','REG_MED','REG_MED_I','REG_MED_N','REG_MED_P','ENS_ESP','ESP_IN_CR','ESP_IN_PE',
           'ESP_FU_8A','ESP_FU_9A','ESP_MED','ESP_MED_I','ESP_MED_N','ESP_MED_P','ESP_EJA_F','ESP_EJA_M','EJA',
           'EJA_FUND','EJA_MED','EJA_PROJ']
esc_rmsp = esc_rmsp.drop(col_rmsp, axis = 1)

C:\Users\giova\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [52]:
#verificando o tamanho
print ( 'Dados escolares' + str(dados_escolares.shape))
print ( 'Rendimento' + str(rendimento.shape))
print ( 'ESC_RMSP' + str(esc_rmsp.shape))
print( 'Nota' + str(nota.shape))

Dados escolares(104292, 18)
Rendimento(14944, 13)
ESC_RMSP(10937, 96)
Nota(1243, 3)


In [53]:
#Unificando os dados de vários anos
dados_escolares = dados_escolares.groupby('CODMEC').mean()
rendimento = rendimento.groupby('CODMEC').mean()
esc_rmsp = esc_rmsp.groupby('CODESC').mean()

In [54]:
#verificando o tamanho
print ( 'Dados escolares' + str(dados_escolares.shape))
print ( 'Rendimento' + str(rendimento.shape))
print ( 'ESC_RMSP' + str(esc_rmsp.shape))
print( 'Nota' + str(nota.shape))

Dados escolares(9642, 14)
Rendimento(4892, 12)
ESC_RMSP(10937, 93)
Nota(1243, 3)


In [55]:
#Apagando a classe desbalanceada
nota = nota[nota.Classificacao != 0]

In [56]:
#Em todos Dataframes temos mais escolas do que o nosso target
dados_escolares = dados_escolares.reset_index()[dados_escolares.reset_index().CODMEC.isin(nota['CODIGO DA ENTIDADE'])]
rendimento = rendimento.reset_index()[rendimento.reset_index().CODMEC.isin(nota['CODIGO DA ENTIDADE'])]
esc_rmsp = esc_rmsp.reset_index()[esc_rmsp.reset_index().CODESC.isin(nota['CODIGO DA ENTIDADE'])]

In [57]:
#verificando o tamanho
print ( 'Dados escolares' + str(dados_escolares.shape))
print ( 'Rendimento' + str(rendimento.shape))
print ( 'ESC_RMSP' + str(esc_rmsp.shape))
print( 'Nota' + str(nota.shape))

Dados escolares(692, 15)
Rendimento(504, 13)
ESC_RMSP(1241, 94)
Nota(1241, 3)


In [58]:
#apagando as linha com valor NaN
dados_escolares.dropna(inplace = True, how = 'any')

In [59]:
esc_rmsp.drop(esc_rmsp.columns[esc_rmsp.isnull().mean() > 0.15], axis=1, inplace = True)
esc_rmsp.dropna(how='any', axis = 0, inplace = True)

In [60]:
#verificando o tamanho
print ( 'Dados escolares' + str(dados_escolares.shape))
print ( 'Rendimento' + str(rendimento.shape))
print ( 'ESC_RMSP' + str(esc_rmsp.shape))
print( 'Nota' + str(nota.shape))

Dados escolares(690, 15)
Rendimento(504, 13)
ESC_RMSP(931, 21)
Nota(1241, 3)


In [75]:
nota.rename(columns = {'CODIGO DA ENTIDADE':'CODMEC'}, inplace = True)

In [88]:
new_df = pd.merge(dados_escolares, nota, on = 'CODMEC')

In [91]:
new_df.drop(['CODMEC', 'ANO','NUMERO DE PARTICIPANTES'], axis = 1, inplace = True)
new_df.columns

Index(['Qtde_Docentes', 'NUM_FUNC', 'CL_CRECHE', 'CL_PRE', 'AL_CRECHE',
       'AL_PRE', 'CLEM', 'ALEM', 'CLFUND', 'ALFUND', 'TOTALEJA', 'TOT_EE',
       'TOTAL_ALUNOS', 'Classificacao'],
      dtype='object')

In [92]:
data = new_df.drop('Classificacao', axis = 1)
target = new_df.Classificacao

In [115]:
lm = LinearRegression()
scaler = StandardScaler()
data = scaler.fit_transform(data)
pca = PCA(0.95)
data = pca.fit_transform(data)
kf = KFold(5, shuffle = True)
for train_i, test_i in kf.split(data):
    x_train, x_test = data[train_i], data[test_i]
    y_train, y_test = target[train_i], target[test_i]
    lm.fit(x_train, y_train)
    print('-', lm.score(x_train, y_train))
    print(lm.score(x_test, y_test))

- 0.4017775339450078
0.32669680375422194
- 0.3647245434379016
0.46933406951145606
- 0.4149241888977693
0.22617913361313455
- 0.39421595408007537
0.3719938282869293
- 0.3944567361106862
0.37489445455461556


In [113]:
target.value_counts()

2    492
3     84
1     74
4     40
Name: Classificacao, dtype: int64